In [1]:
%%capture
# %pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

In [2]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline,
    MBartForConditionalGeneration, MBart50TokenizerFast,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftConfig,
    PeftModel,
    AutoPeftModelForCausalLM
)
import os, torch
# from datasets import load_dataset
import time

2024-06-25 12:45:26.055196: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 12:45:26.055294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 12:45:26.194818: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
lang_detect_name = 'qanastek/51-languages-classifier'
lang_detect_tokenizer = AutoTokenizer.from_pretrained(lang_detect_name)
lang_detect_model = AutoModelForSequenceClassification.from_pretrained(lang_detect_name).to('cuda:0')
classifier = TextClassificationPipeline(model=lang_detect_model, tokenizer=lang_detect_tokenizer)

translate_name = "facebook/mbart-large-50-many-to-many-mmt"
translate_model = MBartForConditionalGeneration.from_pretrained(translate_name).to('cuda:0')
translate_tokenizer = MBart50TokenizerFast.from_pretrained(translate_name)

# Load base model(Zephyr-7B 7B)
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

# gallantVN/zephyr-chat-pizza-1
chat_tokenizer = AutoTokenizer.from_pretrained("gallantVN/zephyr-chat-pizza-1")
chat_model = AutoPeftModelForCausalLM.from_pretrained(
    "gallantVN/zephyr-chat-pizza-1",
#     low_cpu_mem_usage=True,
    return_dict=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,)

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.63k [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [4]:
def lang_detect(prompt):
    """
    Dectect which language that user used for chat

    Returns:
        Language tag
    """
    res = classifier(prompt)
    tag = res[0]["label"].replace("-", "_")
    return tag

def translate(tag_inp, tag_out, prompt):
    """
    Translate from user's language to English and vice versa.

    Returns:
        English prompt or answer in user's language.
    """
    translate_tokenizer.src_lang = tag_inp
    encoded_prompt = translate_tokenizer(prompt, return_tensors="pt").to('cuda:0')
    generated_tokens = translate_model.generate(
        **encoded_prompt,
        forced_bos_token_id=translate_tokenizer.lang_code_to_id[tag_out]
    )
    ans = translate_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return ans[0]

def assistant(tag_inp, tag_out, ques):
    dict_tag = {
        "en": "XX", # english
        "es": "XX", # spanish
        "fr": "XX", # french
        "ja": "XX", # japanese
        "nl": "XX", # dutch
    }
    try:
        split_tag_inp = tag_inp.split("_")
        tag_inp = f"{split_tag_inp[0]}_{dict_tag[split_tag_inp[0]]}"
#         print(tag_inp)
    except: pass
    
    if tag_inp == "en_XX":
        que = ques
        # chat model
        messages.append({"role":"user", "content": que})
        tokenized_chat = chat_tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
        outputs = chat_model.generate(tokenized_chat.to('cuda:1'), max_new_tokens=128)
        ans = chat_tokenizer.decode(outputs[0])
        anss = ans.split("\n<|assistant|>\n")[-1].split("</s>")[0]
        
        # output
        messages.append({"role": "assistant", "content": anss})
        print("\nAssistant:", anss)
        ends = time.time() - start
        print(ends)
    else:
        que = translate(tag_inp, tag_out, ques)
        tag_out = tag_inp
        tag_inp = "en_XX"
        
        # chat model
        messages.append({"role":"user", "content": que})
        tokenized_chat = chat_tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
        outputs = chat_model.generate(tokenized_chat.to('cuda:1'), max_new_tokens=128)
        ans = chat_tokenizer.decode(outputs[0])
        ans = ans.split("\n<|assistant|>\n")[-1].split("</s>")[0]
        anss = translate(tag_inp, tag_out, ans)

        # output
        messages.append({"role": "assistant", "content": anss})
        print("\nAssistant:", anss)
        ends = time.time() - start
        print(ends)

In [5]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [8]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot assistant for ordering pizza at KinPizza.",
    },
#     {"role": "user", "content": "Hi, I'm interested in ordering a pizza from KinPizza. What types of pizzas do you offer?"},
 ]

ans = ''
while True:
    que = input("User: ")
    if que == "Bye":
        break
    
    # find lang tag
    start = time.time()
    tag_inp = lang_detect(que)
    split_tag_inp = tag_inp.split("_")
    tag_inp = f"{split_tag_inp[0]}_{split_tag_inp[1]}"
    tag_out = "en_XX"
    
    # assis
    assistant(tag_inp, tag_out, que)

User:  Hi, I'm interested in ordering a pizza from KinPizza. What types of pizzas do you offer?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Assistant: Welcome to KinPizza! We have a wide variety of pizzas to choose from. Our most popular choices include:
 Cheese Pizza: A classic combination of cheese and cream sauce, served with honey.
 Chicken BBQ Pizza: A savory blend of cheese, tomato sauce, chicken, onion, bell pepper, and BBQ sauce.
 Veggie Pizza: A healthy option with cheese, tomato sauce, onion, bell pepper, mushroom, and black olive.
 Sausage Pizza: A meaty delight with cheese, tomato sauce, and sausage.
11.134695768356323


User:  tôi muốn đặt cheese pizza


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


Assistant: Giá của pizza phô mai trung bình là 179,000 VND.
4.7033655643463135


User:  Thank you, here is my phone number:0919512867


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Assistant: Perfect! Your total cost is 179,000 VND. Your pizza will be delivered within 30 minutes.
5.364912986755371


User:  Bye


In [9]:
messages

[{'role': 'system',
  'content': 'You are a friendly chatbot assistant for ordering pizza at KinPizza.'},
 {'role': 'user',
  'content': "Hi, I'm interested in ordering a pizza from KinPizza. What types of pizzas do you offer?"},
 {'role': 'assistant',
  'content': 'Welcome to KinPizza! We have a wide variety of pizzas to choose from. Our most popular choices include:\n Cheese Pizza: A classic combination of cheese and cream sauce, served with honey.\n Chicken BBQ Pizza: A savory blend of cheese, tomato sauce, chicken, onion, bell pepper, and BBQ sauce.\n Veggie Pizza: A healthy option with cheese, tomato sauce, onion, bell pepper, mushroom, and black olive.\n Sausage Pizza: A meaty delight with cheese, tomato sauce, and sausage.'},
 {'role': 'user', 'content': 'I wanted to put cheese pizza.'},
 {'role': 'assistant',
  'content': 'Một lựa chọn tuyệt vời! Pizza phô mai của chúng tôi có 3 kích cỡ: Trung (24cm), to (34cm), và to (42cm).'},
 {'role': 'user', 'content': 'Give me an average 